This notebook is for setting up the chipwhisperer nano and capturing a balanced trace used for training and testing. We create a hdf5 file where we store the data.

We run this in the SCA101 folder for chipwhisperer for Windows.

In [17]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

In [18]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [19]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../firmware/mcu/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

Building for platform CWNANO with CRYPTO_TARGET=TINYAES128C
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
Blank crypto options, building for AES128
arm-none-eabi-gcc (15:13.2.rel1-2) 13.2.1 20231009
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CWNANO 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
.
.
.
Compiling:
Compiling:
Compiling:
Compiling:
Compiling:
Compiling:
Compiling:
-en     simpleserial-aes.c ...
-en     .././simpleserial/simpleserial.c ...
.
-en     .././hal/hal.c ...
-en     .././hal//stm32f0_nano/stm32f0_hal_nano.c ...
-en     .././crypto/tiny-AES128-C/aes.c ...
-en     .././hal//stm32f0/stm32f0_hal_lowlevel.c ...
-en     .././crypto/aes-independant.c ...
Assembling: .././hal//stm32f0/stm32f0_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -x assembler-with-cpp -mthumb -mfl

/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/closer.c:47:(.text+0xc): warning: _close is not implemented and will always fail
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-lseekr.o): in function `_lseek_r':
/build/newlib-38V0JC/newlib-4.4.0.20231231/build_nano/arm-none-eabi/thumb/v6-m/nofp/newlib/../../../../../../newlib/libc/reent/lseekr.c:49:(.text+0x10): warning: _lseek is not implemented and will always fail
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-readr.o): in function `_read_r

/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-closer.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-lseekr.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-readr.o): note: the message above does not take linker garbage collection into account
/usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/bin/ld: /usr/lib/gcc/arm-none-eabi/13.2.1/../../../arm-none-eabi/lib/thumb/v6-m/nofp/libg_nano.a(libc_a-writer.o): note: the message above does not take linker garbage collection into account
-e Do

In [20]:
cw.program_target(scope, prog, "../../../firmware/mcu/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F03xx4/03xx6
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6051 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6051 bytes


In [21]:
#Testing different capture methods, to see if it has any effect

In [22]:
#FROM LAB3_3
from tqdm.notebook import trange
import numpy as np
import time

ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)

N = 1
for i in trange(N, desc='Capturing traces'):
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        continue
    
    response = target.simpleserial_read('r', 16)
    
    trace_array.append(scope.get_last_trace())
    textin_array.append(text)
    
    key, text = ktp.next() 
    

Capturing traces:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
cw.plot(trace_array[0])


:Curve   [x]   (y)

In [7]:
print(textin_array[0])

CWbytearray(b'3e b2 27 f9 d9 55 e7 16 6b e9 d3 5c 69 d3 0b 43')


In [24]:
#testing with just capture trace method
trace1_array=[]
textin1_array=[]
trace = cw.capture_trace(scope, target, text, key, as_int = True)
trace2 = cw.capture_trace(scope, target, text, key, as_int = True)
trace1_array = trace.wave
textin1_array = trace.textin

In [25]:
cw.plot(trace.wave)*cw.plot(trace2.wave)
#print(textin1_array)

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [26]:
#CODE IS FROM scaaml.scaaml.capture.input_generators.input_generators.py

from typing import Iterator, List, Optional

import numpy as np
import numpy.typing as npt

InputGeneratorT = npt.NDArray[np.int64]


def single_bunch(length: int,
                 elements: int = 256,
                 seed: Optional[int] = None) -> InputGeneratorT:
    """Generate a single bunch of arrays (2D array -- rows * columns). Each row
    has elements chosen uniformly independently from range(elements). But two
    rows have no index where they are equal.

    Rationale: neural networks are very good at picking statistical properties.
    By random chance the value 0x2A of byte 3 (concrete values chosen for
    illustration purposes only) appears more often than others and this is
    consistent in train and valuation set. Then we can get false positives (it
    looks like the neural network is sort of working). Generating the train set
    such that each byte attains each value the same number of times helps to
    mitigate this situation.

    Explanation of the algorithm: for each index we create a random permutation
    of elements and concatenate those, we then return this transposed.

    Beware of doing two for loops over two single_bunch outputs generated with
    the same seed. This would return identical arrays.

    Args:
      length (int): Length of each of the resulting arrays.
      elements (int): Each array contains numbers in `range(elements)`.
      seed (Optional[int]): Seed for the random generator. Passed to
        numpy.random.default_rng. For testing purposes only.

    Returns: an np.array of shape `(elements, length)` where for each index
      `i in range(length)` and for each `value in range(elements)` there is an
      array which contains `value` on position `i`.

    Example use:
      >>> print("Small and deterministic example:")
      >>> single_bunch(5, 3, 42)
      array([[2, 0, 1, 2, 2],
             [1, 2, 2, 0, 0],
             [0, 1, 0, 1, 1]])
      >>> print("Realistic example:")
      >>> for value in single_bunch(32):
      >>>     print(f{value} has each byte chosen uniformly at random)
      >>> print("But no two values had same element on the same position.")
    """
    rng: np.random.Generator = np.random.default_rng(seed=seed)

    transposed_result: List[InputGeneratorT] = []

    for _ in range(length):
        # One column (content of result[:, i]).
        column: InputGeneratorT = np.arange(elements, dtype=np.uint8)

        # permuted does a copy, just to be sure rows are not constant.
        transposed_result.append(rng.permuted(column))

    # Convert to np
    result: InputGeneratorT = np.array(transposed_result, dtype=np.uint8)

    # Transpose the output to get a list of arrays
    result = np.transpose(result)

    return result


def balanced_generator(length: int,
                       bunches: int = 1,
                       elements: int = 256) -> Iterator[InputGeneratorT]:
    """Generator of values for the training set. Feel free to use
    unrestricted_generator for holdout, but not for training split.

    Args:
      length (int): Length of each yielded array.
      bunches (int): How many bunches to return (yields `bunches * elements`
        random arrays).
      elements (int): Each array contains numbers in `range(elements)`.

    Example use: Output may vary based on randomly generated numbers.

    Two bunches examples (generates bunches * elements arrays):
    >>> for value in balanced_generator(5, bunches=2, elements=3):
    >>>     print(value)
    [0 1 2 2 1]
    [1 2 1 0 2]
    [2 0 0 1 0]
    [1 1 0 1 1]
    [2 0 2 2 2]
    [0 2 1 0 0]
    >>> print("Each value repeats the same number of times in each column.")
    >>> print("But columns may repeat.")

    AES128 example: for each key have balanced plain_texts (thus SBOX inputs are
    also balanced). Note that length is number of bytes = 16 (128 bits / 8).
    This results in 65,536 traces, 256 different keys and for each key 256
    different plain_texts (for different keys we generate different plain_texts
    at random).
    >>> for key in balanced_generator(length=16):
    >>>     for plain_text in balanced_generator(length=16):
    >>>         print(f"Capture trace with {key = } and {plain_text = }")

    AES128-like example (small, output may be different, based on random
    chance):
    >>> for key in balanced_generator(length=5, elements=3):
    >>>     for plain_text in balanced_generator(length=5, elements=3):
    >>>         print(f"Capture trace with {key = } and {plain_text = }")
    key = array([0, 2, 0, 0, 0]) plain_text = array([0, 1, 1, 1, 0])
    key = array([0, 2, 0, 0, 0]) plain_text = array([2, 2, 0, 2, 2])
    key = array([0, 2, 0, 0, 0]) plain_text = array([1, 0, 2, 0, 1])
    key = array([2, 1, 1, 2, 2]) plain_text = array([2, 0, 0, 2, 2])
    key = array([2, 1, 1, 2, 2]) plain_text = array([1, 1, 2, 0, 1])
    key = array([2, 1, 1, 2, 2]) plain_text = array([0, 2, 1, 1, 0])
    key = array([1, 0, 2, 1, 1]) plain_text = array([1, 0, 2, 0, 0])
    key = array([1, 0, 2, 1, 1]) plain_text = array([2, 2, 1, 1, 2])
    key = array([1, 0, 2, 1, 1]) plain_text = array([0, 1, 0, 2, 1])

    Get numpy array of all results:
    >>> np.array(list(balanced_generator(5, bunches=2, elements=3)))
    """
    for _ in range(bunches):
        bunch: InputGeneratorT = single_bunch(
            length=length,
            elements=elements,
            seed=None,  # random
        )

        # yield each row
        yield from bunch


def unrestricted_generator(length: int,
                           bunches: int = 1,
                           elements: int = 256) -> Iterator[InputGeneratorT]:
    """Do not use this for the training set, use balanced_generator instead.
    Each element is chosen uniformly at random independently from others.

    Args:
      length (int): Length of each yielded array.
      bunches (int): How many bunches to return (yields `bunches * elements`
        random arrays).
      elements (int): Each array contains numbers in `range(elements)`.

    Example use: there is a single bunch, but the first two arrays have the
    same value of the last element. Check balanced_generator where this cannot
    happen. Output may vary based on randomly generated numbers.
    >>> for value in unrestricted_generator(5, elements=3):
    >>>     print(value)
    [0 1 1 1 1]
    [1 0 2 0 1]
    [2 1 1 2 0]
    """
    rng: np.random.Generator = np.random.default_rng(seed=None)

    for _ in range(bunches * elements):
        yield rng.integers(low=0, high=elements, size=length, dtype=np.int64)


In [13]:
!pip install h5py


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


This part of the notebook details the creation of our file. We manually set the amount of traces we want to collect, and the name of the dataset.

In [41]:
import h5py
import numpy as np
import time
import os
from tqdm.notebook import tnrange
from binascii import hexlify
import matplotlib.pyplot as plt

NUM_TRACES = 65536
project_name = "testing_set"
implementation = os.path.basename('../../../hardware/victims/firmware/simpleserial-aes')
path = f"traces/{implementation}-{NUM_TRACES}-diff{''}.hdf5"


f = h5py.File(project_name+".hdf5", 'w')

f.attrs["trace_type"] = "profile"
f.attrs["data_type"] = "plaintexts different"
f.attrs["implementation"] = implementation
f.attrs["adc_samples"] = scope.adc.samples

trace_set = f.create_dataset("trace", (NUM_TRACES, scope.adc.samples), dtype=np.int16)
plaintext_set = f.create_dataset("data", (NUM_TRACES, 16), dtype=np.uint8)
key_set = f.create_dataset("key", (NUM_TRACES, 16), dtype=np.uint8)

if "mask" in implementation:
    mask_group = f.create_group("mask")
    keymask_set = mask_group.create_dataset("key", (NUM_TRACES,2), dtype=np.uint32)
    datamask_set = mask_group.create_dataset("data", (NUM_TRACES,), dtype=np.uint32)


This cell is where the traces are captured and stored.
We define how many keys we want, and how many traces for each key.
We create a list of plaintexts and keys before entering the loop.
In the loop we generate a new list of plaintext for very key.


In [42]:
NUM_KEYS = 256
NUM_PT_PER_KEY = 256
key_idx  = 0
pt_idx = 0

keys = np.array(list(balanced_generator(length=16)))
pts = (np.array(list(balanced_generator(length=16))))

#We will use capture trace, and scale the traces later
for i in tnrange(NUM_TRACES, desc = "Capturing traces"):
    if i % NUM_KEYS == 0 and i != 0:
        key_idx+=1
        pt_idx=0
        key = cw.bytearray(keys[key_idx])
        pts = np.array(list(balanced_generator(length=16)))
        #print("hei")
    trace = None
    while trace is None:
        pt = cw.bytearray(pts[pt_idx])
        pt_idx+=1
        #print("pt",pt)
        #print("key",key)
        trace = cw.capture_trace(scope, target, pt, key, as_int=True)
        #checking key
    
    #if cw.bytearray(key_set[key_idx]) != trace.key:
     #   print(f"error, wrong key{key_set[key_idx]} does not match {trace.key[:16]}")
    if pt == trace.textin:
        plaintext_set[i] = np.array(trace.textin[:16])
    elif pt != trace.textin:
        print(f"pt does not match, in {pt}, out {trace.textin}")
        break;
    
    if key == trace.key:
        key_set[i] = np.array(trace.key[:16])
    elif key != trace.key:
        print("key does not match")
        break;  
    
    trace_set[i] = trace.wave

Capturing traces:   0%|          | 0/65536 [00:00<?, ?it/s]

hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
hei
